In [1]:
import wandb
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    LabelEncoder,
    Normalizer,
)
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV,RidgeClassifier,RidgeClassifierCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier,VotingClassifier,BaggingClassifier,RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier,XGBRFClassifier
from catboost import CatBoostClassifier,CatBoost
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
np.random.seed(42)

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
X = data.drop('fetal_health',axis=1)
y = data['fetal_health']

In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [5]:
PROJECT_NAME = 'Fetal-Health-Clf'

In [6]:
def valid(model,X,y,val=False):
    preds = model.predict(X)
    if val is True:
        results = {
        'Val Accuracy':accuracy_score(y,preds),
        'Val Precision':precision_score(y,preds,average='macro'),
        'Val F1 Score':f1_score(y,preds,average='macro'),
        'Val Recall':recall_score(y,preds,average='macro')
        }
    else:
        results = {
            'Accuracy':accuracy_score(y,preds),
            'Macro Precision':precision_score(y,preds,average='macro'),
            'F1 Score':f1_score(y,preds,average='macro'),
            'Recall':recall_score(y,preds,average='macro')
        }
    return results
def fit(model,X_train,X_test,y_train,y_test,name):
    print(len(X_train.columns))
    wandb.init(project=PROJECT_NAME,name=name)
    model = model()
    model.fit(X_train,y_train)
    wandb.log(valid(model,X_train,y_train))
    wandb.log(valid(model,X_test,y_test,True))
    wandb.sklearn.plot_calibration_curve(model, X, y, name)
    wandb.finish()

In [8]:
# fit(KNeighborsClassifier,X_train,X_test,y_train,y_test,'baseline')

In [9]:
def fe(data,col):
    max_num = data[col].quantile(0.95)
    min_num  = data[col].quantile(0.05)
    data = data[data[col] < max_num]
    data = data[data[col] > min_num]
    return data

In [ ]:
for col list(X.columns):
    new_data = fe(X,col)